Install litenai. It would install all the required components.

In [ ]:
!pip install --quiet litenai-0.0.53-py3-none-any.whl

For colab notebooks, install holoviews and bokeh

In [ ]:
!pip install --quiet jupyter_bokeh
!pip install --quiet holoviews==1.16

Clone sample logfiles into /content directory.

In [ ]:
!git clone https://github.com/litenai/samplelogfiles

After cloning go to Files on left pane, open /content/samplelogfiles/litenai-config.yml by clicking on it, and update with a valid OPENAI API Key.

Once the yaml file is changed, there is no need to do it again if rerunning the collab notebooks

Set JAVA_HOME and SPARK_HOME for proper spark operations. Set LITEN_CONFIG_FILE to the modified liten.yaml

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["LITEN_CONFIG_FILE"] = "/content/samplelogfiles/litenai-config.yml"

Import required libraries to run liten server.

In [ ]:
import os
# import pandas
import pandas as pd
# import spark libs
import pyspark
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, TimestampType
# import plot libs
import panel
import hvplot
import hvplot.pandas
%env HV_DOC_HTML=true

In [ ]:
import litenai

This s

In [ ]:
server = litenai.start_server(port=80, address="0.0.0.0")

In [ ]:
!pip install pyngrok --quiet
from pyngrok import ngrok
# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "<add your ngrok token here>"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 80 for http://localhost:80
public_url = ngrok.connect(addr="80")
print("Tracking URL:", public_url)

You can now stop the server.

In [ ]:
server.stop()

In this example. We show how we can use litenai using a single send API command. send orchestrates the agent to complete the prompts.

In [ ]:
config = litenai.Config()
ten = litenai.Session.get_or_create(name='test', config=config)

Run sample spark, pandas and plots to ensure that setup is all good. Run a sample queries on emailaccesslog table.

In [ ]:
print(f"Print a few lines")
df = ten.spark.sql("select * from emailaccesslog")
df.show(5)
print(f"Total number of log lines")
cntDf = ten.spark.sql("select count(*) from emailaccesslog")
cntDf.show()
print(f"Request counts which were redirected")
st3xxDf = ten.spark.sql("SELECT Status, COUNT(*) FROM emailaccesslog WHERE Status LIKE '2%%' GROUP BY Status")
st3xxDf.show(5)

In [ ]:
st17Df = ten.spark.sql("SELECT * FROM emailaccesslog")
pdf = st17Df.toPandas()
pdf.head()

In [ ]:
hvplot.extension('bokeh')
plot = pdf.hvplot.scatter(x='status',y='latency',title='Latencies for various status')
plot

These log tables are loaded as part of log analysis.<br>
*`emailserverlog`* Table of email access server log<br>
*`syslog`* - Lunux system log file<br>
*`ipnetlog`* - network ip log file<br>
In the following cells, we will show how Liten can be used to observe and reason the data out.

send is one API to talk with Liten Chat. From the prompt, Liten orchestrates the right agent to complete the action.

In [ ]:
ten.send("Can you answer this general question? Could you tell me about server status codes? What is the status code for internal errors")

In [ ]:
ten.send('Can you generate SQL for the following descriptionfor emailaccesslog table name. select all rows where status has internal server error codes?')

You can exexute this sql like this.

In [ ]:
ten.send("""Execute this sql code.
```
SELECT *
FROM emailaccesslog
WHERE status >= 500 AND status < 600
```""")

Analyze can be used to analyze a given data.

In [ ]:
ten.send("Analyze the data to tell me the causes of internal server errors? Also tell me how they can be fixed")

Liten chatbot which can be launched as a cell in jupyter jupyter.  It looks at the data in memory and provides a way to chat with the data.
Enterprise Liten product connects with a lakehouse data base in S3, and lets you analyze your data.
You can launch the chatbot here. There are a few chatbot examples provided in the notes below for you to try it out.

In [ ]:
hvplot.extension('bokeh')
panel.extension('tabulator')
chatbot= liten.ChatBot(ten)
chatbot.start()

These are a few more example chats after this also that you can try.

#### Status Code Analysis
What status codes did the service see in a given time frame. Can we aggregated and view.
#### Suggested Chat Flow
Select emailaccesslog table from the menu </br>
```
User: Add temp to memdata
```
From dropdown box select temp
```
User: Generate sql code to select 100 rows from the table
Bot: SELECT * FROM emailaccesslog LIMIT 100;
User: Execute this sql
'''
SELECT * FROM emailaccesslog LIMIT 100;
'''
```
Liten will execute the sql and store result in temp. It would also show a few lines of the result. You can now look at the plots
```
User: plot temp
```
In the interactive plot, select X=cpuLatency Y=payloadSize groupby status. You can visualize the results in an interactive way.
You can also ask Liten to analyze the data
```
User: Analyze the data to find out for any status code errors. Can you also tell me how I can fix these.
```
Bot will provide an analysis.


Now you can try a bit more complex SQL
```
User: Generate sql for the following prompt.
How many 200, 300, 400, 500 status codes did the service see in last one year.
```
Bit will print out a SQL code. Now execute this
```
User: Execute this sql
'''
SELECT status, COUNT(*) as count FROM emailaccesslog WHERE
time >= date_trunc('year', current_timestamp()) - interval '1 year'
GROUP BY status HAVING status IN (200, 300, 400, 500)
'''
```
Chat remembers the context. So, you can now mix it up with general asks.
```
User: Answer this general question.
Can you modify the sql to look between july of 2023 and august of 2023
```
Bot gives a SQL you can now execute this.
```
Execute this sql
'''
SELECT status, COUNT(*) AS count FROM emailaccesslog WHERE time >= timestamp('2023-07-01') AND time < timestamp('2023-09-01') AND status IN (200, 300, 400, 500) GROUP BY status
'''
```
It stores the result in temp. You can plot and view this as well. You can also analyze this data.

### Internal server error analysis
Internal server error analysis for 5xx codes.
#### Suggested chat flow
Select emailaccesslog table
```
User: Answer this general question. Which status codes in server log file indicate internal server errors?
```
You can ask more questions to understand the bot response better.
Now, let us generate sql for some error analysis
```
User: Generate sql to show all rows with 5xx range status code
```
You can also provide multiple steps in the same chat.
```
User: Now do the following. Generate sql to select all rows with status indicating internal server errors. Then, execute the sql. plot the output data next.
```
Bot will provide the sql code. You can execute this.
```
now execute
```
now you can plot.
```
plot this data
```
You can select X=cpuLatency Y=payloadSize group by errorCode to view the plot. It is an interactive plot, so it can be analyzed as needed
You can also analyze this data. This data stores in temp memdata. This is also in the memdata dropdown. We can now analyze this data
```
User: Analyze data to understand more about error codes. For the given error codes, I want to understand more about why it happens and how it happened in the given data.
```
Bot will give a proper analysis and insights about the error in the given data.

Given below are a few more chat sessions that you can try out on server log tables.

### Highest traffic analysis
Which day of the week usually has the highest traffic?

#### Suggested chat flow
Select emailaccesslog table
```
User: Generate sql to show the day of the week which had the highest traffic in last year?
```
Bot generated SQL is soemthing like this. Now you can execute this.
```
'''
SELECT date_trunc('day', time) as day_of_week, COUNT(*) as traffic_count FROM emailaccesslog WHERE time >= date_sub(current_date(), 365) GROUP BY day_of_week ORDER BY traffic_count DESC LIMIT 1;
'''
```
```
Execute the generated sql
```
Now we can modify it further to drill down into the code.
```
User: Answer this general question. Modify the sql code to pick top 3 days of the week with highest traffic counts?
```
Bot can now generate the modified SQL.
```
'''
SELECT date_trunc('day', time) as day_of_week, COUNT(*) as traffic_count FROM emailaccesslog WHERE time >= date_sub(current_date(), 365) GROUP BY day_of_week ORDER BY traffic_count DESC LIMIT 3;
'''
```
Now you can execute this.
```
Execute the last generated sql
```

#### Highest traffic days
Which time of day has most traffic?
#### Suggested chat flow
Select emailaccesslog table.
```
User: generate sql code to show the time of day with the most traffic. Use hour long time slots to get the traffic data.
```
Pick Bot generated SQL to run
```
User: execute this sql code.
'''
SELECT date_trunc('hour', time) AS time_slot, COUNT(*) AS traffic_count FROM emailaccesslog GROUP BY time_slot ORDER BY traffic_count DESC LIMIT 1
'''
```
More analysis can be done on this data.
```
User: generate sql to show the  time of the day has most traffic. You can group time in 5 minutes chunk.
```
You can execute this as well
```
User: execute this sql
'''
SELECT date_trunc('hour', time) AS hour,
       floor((date_part('minute', time) / 5)) * 5 AS minute_chunk,
       COUNT(*) AS traffic_count
FROM emailaccesslog
GROUP BY hour, minute_chunk
ORDER BY traffic_count DESC
LIMIT 1;
'''
```

### Traffic flow variations
How much did traffic increase or decrease in last 12 months.
#### Suggested chat flow
Select emailaccesslog table.
```
User: generate sql to show the following. How much did traffic increase or decrease in last 12 months.
...
...

User: execute the following sql code
'''
SELECT
  SUM(CASE WHEN DATE_TRUNC('month', time) >= DATE_TRUNC('month', CURRENT_DATE - INTERVAL 12 MONTH)
           THEN 1 ELSE -1 END) AS traffic_change
FROM
  emailaccesslog;
'''
```



### Analyze all errors
analyze all errors in emailaccesslog file
#### Suggested chat flow
Select emailaccesslog table.
```
User: generate sql to select all rows with errors
....
User: execute the following sql code.
'''
SELECT * FROM emailaccesslog WHERE errorCode IS NOT NULL OR detailedErrorCode IS NOT NULL
'''
User: plot
## Look at plot -  select status as X, latency as y, group by errorCode do X as errorCode & latency as Y, group by status
...
User: Can you analyze the data. Please look at the error messages and code in the given data? Can you tell me types of errors occurring. Tell me what can cause it, and how to resolve it. Also tell me how I can avoid these errors.

```


syslog table has some lines from linux system log files. You can use this log file for syslog analysis. Here are a few suggested chat sessions.

### Syslog last x hours analysis
List all errors from syslog in last 'x' hours/days and suggest their solutions.
Example: User is trying to install nginx but it errors out. Check log files and search for them on stackoverflow?

#### Suggested chat flow
Select syslog table

```
User: If there is error or fail in message column it means that there was an error. Write a  sql query to select only these rows.<br>
...
User: Execute the following sql
'''
SELECT * FROM syslog WHERE message LIKE '%error%' OR message LIKE '%fail%'<br>
'''
User: Can you analyze this data? I want to group the types of error being seen. In addition, suggest a solution as well.
...
You can see and analyze the data. You can also plot the data to see the errors and their frequency.


#### Linux syslog CRC error files
Analyze CRC errors from system log files
- Print all lines in syslog where you see an error. Print 4 or 5 lines before and after
- Across all syslogs, identify when how many times RAM is shown CRC errors.
- Write a script to read syslog and declare RAM corrupted if you see more than 10 errors in 24 hour time perod.

#### Suggested chat flow
Select syslog table
```
User: If there is error or fail in message column it means that there was an error. Write a query to select only these rows. Get an aggregated count grouped by the message column.
...
User: Execute the sql
'''
SELECT message, COUNT(*) AS error_count
FROM syslog
WHERE message LIKE '%error%' OR message LIKE '%fail%'
GROUP BY message;

SELECT *
FROM syslog
WHERE message LIKE '%fail%'
'''
You can also analyze this data now.
```
User: Analyze the data to tell me the types of authentication failures you see in the data? If possible tell me why it can happen and what can i do to avoid it?
```
...
User: Answer this general question. In linux system log files, how can you know if RAM has had CRC errors. What errors are expected in the log file messages?
....
User: Please analyze data block. By looking at the linux log data, can you tell if there has been CRC errors in RAM.
...

...
User: Declare RAM corrupted if you see more than 10 errors in 24 hour time period. Write a python script to do that.
...
```
These are a few sample user queries given above.

### Network log analysis
A sample network log file is loaded as ipnetlog file.

Various analysis can be done on this data using the chat. Some samples include
* Newly opened TCP connections for hist
* SuperSpreader	Hosts that contact more than threshold unique destinations
Port Scan	Hosts that send traffic over more than threshold destination ports.
* DDoS	Hosts that receive traffic from more than threshold unique sources.
Syn Flood	Hosts for which the number of half-open TCP connections exceeds threshold Th.
* Completed Flow	Hosts for which the number of incomplete TCP connections exceeds threshold.
* Slowloris Attack	Hosts for which the average transfer rate per flow is below threshold.
* DNS Tunneling	Hosts for which new TCP connections are not created after DNS query.
* Zorro Attck	Hosts that receive “zorro” command after telnet brute force.
* Reflection DNS	Hosts that receive DNS response of type RRSIG from many unique senders without requests.
* Dns ttl tracking	Track TTL for DNS resource record names



### Heavy Hitter
Largest payload flows
#### Suggested chat flow
Select ipnetlog file.
```
User: Generate sql to select top 10 rows
...
User: Can you answer this general question? Tell me more about heavy hitters and what to look for to identify the heavy hitters.
..
User: generate sql to do the following. First select top rows with largest payload sizes. aggregate the payloads and group them by source ip address. select top 10 rows. Now select all the rows with all the fields from ipnetlog table having these source ip addresses.
...
User: execute this sql
'''
SELECT *
FROM (
    SELECT ip_src, SUM(CAST(ip_len AS INT)) AS total_payload
    FROM ipnetlog
    GROUP BY ip_src
    ORDER BY total_payload DESC
    LIMIT 10
) AS top_sources
JOIN ipnetlog ON top_sources.ip_src = ipnetlog.ip_src
'''
User: plot temp
...
User: Can you analyze the given data to see the heavy hitters? For these heavy hitters, can you tell if they look suspicious.
```
You can also ask for a high level plan before doing simple steps.
```
User: Please focus on identifying heavy hitters and guide me through the steps?
```

### TCP New Conn
Newly opened TCP connections for hist
#### Suggested chat flow
Select ipnetlog table
```
User: Answer this general question. How do we find a newly opened TCP connection from IP net logs?
...
User: Can you generate the SQL code to show the newly opened TCP connections?
...
User: Execute the following sql
'''
SELECT ip_src, ip_dst, MIN(time) AS earliest_timestamp
FROM ipnetlog
WHERE ip_proto = 'TCP'
GROUP BY ip_src, ip_dst
ORDER BY earliest_timestamp
'''
...
```

###SSH Brute
Number of unique senders sending hosts that similar-sized packets
#### Suggested chat flow
Select ipnetlog table
User can now go ahead and do an analysis.
```
User: Can you plan to understand SSH brute force issues? How should i look at data to be able to do that?
```
User can look at the data observe and analyze it.